In [3]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.4 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 596 kB 85.2 MB/s 
     |████████████████████████████████| 59 kB 9.0 MB/s 
     |████████████████████████████████| 895 kB 92.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import numpy as np
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [12]:
def encoding(question,passage):
  input_encoding = tokenizer.encode(question, passage)
  sep_index = input_encoding.index(tokenizer.sep_token_id)
  num_seg_a = sep_index + 1
  num_seg_b = len(input_encoding) - num_seg_a
  segment_encoding = [0]*num_seg_a + [1]*num_seg_b
  assert len(segment_encoding) == len(input_encoding)
  return segment_encoding, input_encoding
  


In [14]:
def single_answer(question, passage):
    segment_encoding, input_encoding=encoding(question,passage)
    outputs = model(torch.tensor([input_encoding]),token_type_ids=torch.tensor([segment_encoding]),return_dict=True) 
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_encoding)
    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    final_score=(torch.max(start_scores)+torch.max(end_scores))/2
    start_ = np.exp(start_scores.detach().numpy() - np.log(np.sum(np.exp(start_scores.detach().numpy()), axis=-1, keepdims=True)))
    end_ = np.exp(end_scores.detach().numpy() - np.log(np.sum(np.exp(end_scores.detach().numpy()), axis=-1, keepdims=True)))
    start=start_
    end=end_
    if start.ndim == 1:
              start = start[None]
    if end.ndim == 1:
        end = end[None]
    outer = np.matmul(np.expand_dims(start, -1), np.expand_dims(end, 1))
    candidates = np.tril(np.triu(outer), 10 - 1)
    scores_flat = candidates.flatten()
    idx_sort = [np.argmax(scores_flat)]
    starts, ends = np.unravel_index(idx_sort, candidates.shape)[1:]
    final_score = candidates[0, starts, ends]
    return final_score[0],answer

In [16]:
def answer(question,context):
  max_score=-np.inf
  final_answer=""  
  length=len(context)
  start_idx=0
  end_idx=500
  while(end_idx<length):
    current_context=context[start_idx:end_idx]
    score,current_answer=single_answer(question, current_context)
    #print(score, current_answer)
    if score>max_score:
      max_score=score
      final_answer = current_answer
    start_idx+=250
    end_idx+=250
  current_context=context[start_idx:length+1]
  score,current_answer=single_answer(question, current_context)
  #print(score, current_answer)
  if score>max_score:
    max_score=score
    final_answer = current_answer
    
  print(max_score)
  return final_answer
  





In [17]:
context="Lord Robert Clive was appointed the governor of Bengal after the battle of buxar"
question = 'Who was appointed the governor of Bengal after the battle of buxar?'

ans = answer(question, context)
print(ans)

0.99659353
lord robert clive


In [ ]:

file=open("Passage.txt","r")
passage=" "
for line in file:
  passage+=line+" "
#This section enables the user to store the questions in a text file named as "questions.txt" and get the answer. Each answer will take 5 min.
#questions=open("questions.txt","r")
#for question in questions:
  #print(answer(question, passage))

In [19]:

flag=1
while(flag):
  question=input("Enter Your Question: ")  
  if question=="END":
    flag=0
  if flag:
    if question[-1]!="?":
      question+="?"
    print(answer(question, passage))



Enter Your Question: Who was appointed the governor of Bengal after the battle of buxar
0.99640256
mir jafar
Enter Your Question: END
